# Pricing a Swaption using a binomial tree: 

In this module, I price an interest-rate swaption which are options on a swap using the binomial tree. The swaption pricing function has the following inputs:

We price the swap using Martingale pricing formula:

$\large S_t = \mathbb{E}[\frac{S_{t+1} + C_{t+1}}{1 + r_t}]$

and we use the function swap_price (introduced in an earlier module).  

The function swaption_price takes in the following parameters: <br/>
k: the strike of the swaption <br/>
r_f: the fixed rate of the swap <br/>
r_0: the short rate at time 0 <br/>
n_o: the expiry period of the swaption <br/>
n: the expiry period of the swap <br/>
u,d: up and down factors by which the short rate evolves <br/>

To model the short rate, we will use the function short_rate, which models the evolution of the short rate using a binomial tree. 

The swaption pricing is done with the usual risk-neutral method in a binomial tree and by discounting the price using the short-rate of that state.

In [5]:
import numpy as np

In [61]:
def swaption_price(k,r_f,r_0,n_o,n,u,d):
    
    #inputs:
    #k: the strike rate (enter amount in %)
    #r_f: the fixed rate of the swap(enter amount in %)
    #r_0: the short rate at time 0 (enter amount in %)
    #n_o: the expiry period of the swaption
    #n: the expiry period of the swap   
    #u,d: up and down factors by which the short rate evolves
    
    s_price_arr = swap_price(1.0,r_f,r_0,n,u,d) #the swap price on a notional of $1
    
    rate_lattice = short_rate(r_0,n_o,u,d)
    
    rate_lattice/= 100.0
    
    swaption_price = np.zeros((n_o+1,n_o+1))
    
    #price the terminal column of the swaption price lattice
    
    for i in range(n_o+1):
        swaption_price[n_o,i] = max(s_price_arr[n_o,i],0)
        
    q = 0.5    #the risk-neutral probability
      
    for i in range(n_o-1,-1,-1):
        for j in range(i+1):
                swaption_price[i,j] = (1/(1+rate_lattice[i,j]))* \
                                  (q*swaption_price[i+1,j+1]+(1-q)*swaption_price[i+1,j])
                                
        
    return swaption_price[0,0]

In [65]:
swaption_price(0.0,5.0,6.0,3,6,1.25,0.9)

0.06197180915914935

In [66]:
def swap_price(p,k,r_0,n,u,d):
    
    #inputs:
    #p: the notional amount
    #k: the fixed rate (enter amount in %)
    #r_0: the short rate at time 0 
    #n: the expiry period of the swap 
    #u,d: up and down factors by which the short rate evolves
    
    rate_lattice = short_rate(r_0,n-1,u,d)
    
    rate_lattice /= 100.0
    
    swap_price = np.zeros_like(rate_lattice) #create an empty array of same dimension
                                             #as rate_lattice
    
    #price the terminal node of the lattice
    
    for i in range(n):
        swap_price[n-1,i] = (rate_lattice[n-1,i] - k/100.0)/(1+rate_lattice[n-1,i])
        
    #price the remaining nodes using the martingale property and risk-neutral probabilities
    
    q = 0.5    #the risk-neutral probability
      
    for i in range(n-2,-1,-1):
        for j in range(i+1):
                swap_price[i,j] = (1/(1+rate_lattice[i,j]))* \
                                  (q*swap_price[i+1,j+1]+(1-q)*swap_price[i+1,j] + \
                                  (rate_lattice[i,j] - k/100.0))
    return swap_price

In [67]:
def short_rate(r_0,n,u,d):
    
    #S_0 = initial stock price
    #n: number of periods
    #u,d = up/down movement in each step
    
    rate = np.zeros((n+1,n+1))
    rate[0,0] = r_0
    
    for i in range(1,n+1):
        rate[i,0] = rate[i-1,0]*d
        for j in range(1,i+1):
            rate[i,j] = rate[i-1,j-1]*u
        
        
    return rate